The purpose of this script is to retrive the following information from the OpenET API:  
-all 6 models and the Ensemble estimates of ET  
-dates of landsat overpass  
-ET, ETo, ETof, and Precipitation values extracted from SIMS/CIMIS

This script carries out the above operations for both the lettuce and broccoli data using dynamic footprints

Input files:  
Dynamic footprint JSON

Output files:  
-.csv holding all models and Ensemble estimates of ET  
-.csv holding dates of landsat overpass  
-.csv holding SIMS ET as well as ETo, ETof, and Precip values collected via CIMIS

###Mount Drive

In [ ]:
try:
    from google.colab import drive # type: ignore
    drive.mount('/content/home')
except ImportError:
    pass

Drive already mounted at /content/home; to attempt to forcibly remount, call drive.mount("/content/home", force_remount=True).


###Import Dependencies

In [ ]:
import requests
import json
import pandas as pd
import geopandas as gpd
import numpy as np
#from matplotlib.dates import DateFormatter
#import matplotlib.pyplot as plt

###Declare Custom Functions

In [ ]:
def openET_API_polygon_query_ET (MY_API_KEY, START_DATE, END_DATE, INTERVAL, GEOMETRY, UNIT):
  '''
  MY_API_KEY: string
  START_DATE: inclusive, string
  END_DATE: inclusive, string
  INTERVAL: 'daily' or 'monthly'
  GEOMETRY: array of coordinates
  UNIT: 'mm' or 'in'

  '''
  #string of all available models
  models = ['Ensemble','geeSEBAL','SSEBop','SIMS','DisALEXI','PTJPL','eeMETRIC']

  #create df to load up
  API_df = pd.DataFrame()

  for model in models:
    # set your API key before making the request
    header = {"Authorization": MY_API_KEY}

    # endpoint arguments
    args = {
      "date_range": [
        START_DATE,
        END_DATE
      ],
      "interval": 'INTERVAL',
      "geometry": GEOMETRY,
      "model": model,
      "variable": 'ET',
      "reference_et": "CIMIS",
      "reducer": "mean",
      "units": UNIT,
      "file_format": "JSON"
    }

    # query the api
    resp = requests.post(
        headers=header,
        json=args,
        url="https://openet-api.org/raster/timeseries/polygon"
    )

    #print(resp.text)
    intermediate_API_df = pd.read_json(resp.text)
    API_df['date'] = intermediate_API_df['time']
    API_df[model] = intermediate_API_df['et']

    #print some statements to show what step
    print(model, end='')
    print(" done!")

  #return df
  print("dataframe created :)")
  return API_df

In [ ]:
def openET_API_polygon_query_ET_overpass (MY_API_KEY, START_DATE, END_DATE, INTERVAL, GEOMETRY, UNIT):
  '''
  MY_API_KEY: string
  START_DATE: inclusive, string
  END_DATE: inclusive, string
  INTERVAL: 'daily' or 'monthly'
  GEOMETRY: array of coordinates
  UNIT: 'mm' or 'in'

  '''
  #string of all available models
  models = ['Ensemble']

  #create df to load up
  API_df = pd.DataFrame()

  for model in models:
    # set your API key before making the request
    header = {"Authorization": MY_API_KEY}

    # endpoint arguments
    args = {
      "date_range": [
        START_DATE,
        END_DATE
      ],
      "interval": INTERVAL,
      "geometry": GEOMETRY,
      "model": model,
      "variable": 'ET',
      "reference_et": "CIMIS",
      "reducer": "mean",
      "units": UNIT,
      "file_format": "JSON",
      "overpass": "True"
    }

#     # query the api
#     resp = requests.post(
#         headers=header,
#         json=args,
#         url="https://openet-api.org/raster/timeseries/polygon"
#     )

#     #print(resp.text)
#     intermediate_API_df = pd.read_json(resp.text)
#     API_df['date'] = intermediate_API_df['time']
#     API_df[model] = intermediate_API_df['et']

#     #print some statements to show what step
#     print(model, end='')
#     print(" overpass done!")

#   #return df
#   print("dataframe created :)")
#   return API_df

In [ ]:
def openET_API_polygon_query_SIMS_allET (MY_API_KEY, START_DATE, END_DATE, INTERVAL, GEOMETRY, UNIT):
  '''
  MY_API_KEY: string
  START_DATE: inclusive, string
  END_DATE: inclusive, string
  INTERVAL: 'daily' or 'monthly'
  GEOMETRY: array of coordinates
  UNIT: 'mm' or 'in'

  '''
  #string of all available models
  vars = ['ET', 'ETo', 'ETof', 'PR']

  #create df to load up
  API_df = pd.DataFrame()

  for var in vars:
    # set your API key before making the request
    header = {"Authorization": MY_API_KEY}

    # endpoint arguments
    args = {
      "date_range": [
        START_DATE,
        END_DATE
      ],
      "interval": INTERVAL,
      "geometry": GEOMETRY,
      "model": 'SIMS',
      "variable": var,
      "reference_et": "CIMIS",
      "reducer": "mean",
      "units": UNIT,
      "file_format": "JSON"
    }

#     # query the api
#     resp = requests.post(
#         headers=header,
#         json=args,
#         url="https://openet-api.org/raster/timeseries/polygon"
#     )

#     #print(resp.text)
#     intermediate_API_df = pd.read_json(resp.text)
#     API_df['date'] = intermediate_API_df['time']
#     API_df[var] = intermediate_API_df[intermediate_API_df.columns[1]]

#     #print some statements to show what step
#     print(var, end='')
#     print(" done!")

#   #return df
#   print("dataframe created :)")
#   return API_df

###Set Directories and Declare Variables

In [ ]:
MAIN_DIR = '/content/home/MyDrive/B_SanLuis'

#'TRUE' or 'FALSE' depending on whether you would like to export the dataframes as .csv
SAVE_EXPORTS = 'FALSE'

#Choose between 'Lettuce' or 'Broccoli
CROP_OF_CONCERN = 'Broccoli'

#update API key here
MY_KEY = '095Ip2n5f3pJFLh4YHdk6CSdbPezttwadaEgJ5ZHZSSbKXszgLmP51nKIt7i'

if CROP_OF_CONCERN == 'Lettuce':
  START_DATE = '2023-04-19'
  END_DATE = '2023-06-20'
  OVERPASS_START_DATE = '2023-04-11'  # +/-8 days from start/end
  OVERPASS_END_DATE = '2023-06-28'
  GEOMETRY_FILE = MAIN_DIR + '/Data/Lettuce2023Final.geojson'
  print("Geometry source from: ")
  print(GEOMETRY_FILE)
  print("Files will be saved at: N/A")

if CROP_OF_CONCERN == 'Broccoli':
  START_DATE = '2023-07-21'
  END_DATE = '2023-10-04'
  OVERPASS_START_DATE = '2023-07-13'  # +/-8 days from start/end
  OVERPASS_END_DATE = '2023-10-12'
  GEOMETRY_FILE = '/content/home/Shareddrives/WATRS_Field_Data/Publications_Presentations_And_Supporting_Documents/Solymar_et_al_Lettuce_Broccoli_2023/Data/Footprint_Polygons/Broccoli/Broccoli2023_FootprintFinal.json'
  print("Geometry source from: ")
  print(GEOMETRY_FILE)
  print("Files will be saved at: N/A")

if CROP_OF_CONCERN == 'Lettuce2024':
  START_DATE = '2024-04-'
  END_DATE = '2024-06-14'
  OVERPASS_START_DATE = '2024-04-02'  # +/-8 days from start/end
  OVERPASS_END_DATE = '2024-06-22'
  GEOMETRY_FILE = MAIN_DIR + '/Data/Lettuce2024Final.geojson'
  print("Geometry source from: ")
  print(GEOMETRY_FILE)
  print("Files will be saved at: N/A")


Geometry source from: 
/content/home/Shareddrives/WATRS_Field_Data/Publications_Presentations_And_Supporting_Documents/Solymar_et_al_Lettuce_Broccoli_2023/Data/Footprint_Polygons/Broccoli/Broccoli2023_FootprintFinal.json
Files will be saved at: N/A


###Create Coordinate List for OpenET

In [ ]:
FOOTPRINT_GEOM = gpd.read_file(GEOMETRY_FILE)
FOOTPRINT_GEOM_JSON = json.loads(FOOTPRINT_GEOM.to_json())

In [ ]:
MY_COORDS = np.array(FOOTPRINT_GEOM_JSON["features"][0]["geometry"]["coordinates"])

In [ ]:
COORD_LIST = ([])
for i in range(0,MY_COORDS.shape[1]):
  COORD_LIST.append(MY_COORDS[0][i][0])
  COORD_LIST.append(MY_COORDS[0][i][1])

###Run API Queries

In [ ]:
ET_df = openET_API_polygon_query_ET(MY_KEY,START_DATE,END_DATE,'daily',COORD_LIST,'mm')

NameError: name 'openET_API_polygon_query_ET' is not defined

In [ ]:
ET_df

NameError: name 'ET_df' is not defined

In [ ]:
overpass_df = openET_API_polygon_query_ET_overpass(MY_KEY,OVERPASS_START_DATE,OVERPASS_END_DATE,'daily',COORD_LIST,'mm')

In [ ]:
overpass_df

In [ ]:
SIMS_all_df = openET_API_polygon_query_SIMS_allET(MY_KEY,START_DATE,END_DATE,'daily',COORD_LIST,'mm')

In [ ]:
SIMS_all_df

In [ ]:
header = {"Authorization": '095Ip2n5f3pJFLh4YHdk6CSdbPezttwadaEgJ5ZHZSSbKXszgLmP51nKIt7i'}
# endpoint arguments
args = {
  "date_range": [
    '2023-07-21',
    '2023-10-04'
  ],
  "interval": 'daily',
  "geometry": COORD_LIST,
  "model": 'Ensemble',
  "variable": 'ET',
  "reference_et": "CIMIS",
  "reducer": "mean",
  "units": 'mm',
  "file_format": "JSON"
}
# query the api
resp = requests.post(
    headers=header,
    json=args,
    url="https://openet-api.org/raster/timeseries/polygon"
)


In [ ]:
resp.text

'[\n    {\n        "time":"2023-07-21",\n        "et":2.409\n    },\n    {\n        "time":"2023-07-22",\n        "et":2.089\n    },\n    {\n        "time":"2023-07-23",\n        "et":1.635\n    },\n    {\n        "time":"2023-07-24",\n        "et":1.477\n    },\n    {\n        "time":"2023-07-25",\n        "et":1.233\n    },\n    {\n        "time":"2023-07-26",\n        "et":2.811\n    },\n    {\n        "time":"2023-07-27",\n        "et":2.833\n    },\n    {\n        "time":"2023-07-28",\n        "et":2.95\n    },\n    {\n        "time":"2023-07-29",\n        "et":3.093\n    },\n    {\n        "time":"2023-07-30",\n        "et":3.115\n    },\n    {\n        "time":"2023-07-31",\n        "et":2.817\n    },\n    {\n        "time":"2023-08-01",\n        "et":2.458\n    },\n    {\n        "time":"2023-08-02",\n        "et":2.265\n    },\n    {\n        "time":"2023-08-03",\n        "et":2.733\n    },\n    {\n        "time":"2023-08-04",\n        "et":2.403\n    },\n    {\n        "time":